<a href="https://colab.research.google.com/github/AsimPoptani/mhealth-playground/blob/master/Copy_of_Umer_and_Asim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mhealth DataSet

 https://archive.ics.uci.edu/ml/datasets/MHEALTH+Dataset
 

In [0]:
# from google.colab import drive
# drive.mount('/content/drive')

##Imports

In [0]:
%tensorflow_version 2.x

In [0]:
# Imports
# from helper_functions import mhealth_get_dataset
import random
import tensorflow as tf
import numpy as np
from collections import defaultdict

## Get the data

In [0]:
import glob
import urllib.request
import zipfile


# Warning takes 7 seconds to load
# Gives data in a dictionary on the mhealth dataset: [ {id:id,data: 2DArrayOfRawData ,file_location:fileLocation},{id:id,data: 2DArrayOfRawData ,file_location:fileLocation}]
def mhealth_get_dataset(dir_to_files = './mhealth-data/'):
    ''' Returns a dictionary of subjects.

    This DocString is a test to make sure git is working! 
    '''
    
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00319/MHEALTHDATASET.zip'
    filehandle, _ = urllib.request.urlretrieve(url)
    zip_file_object = zipfile.ZipFile(filehandle, 'r')
    files = zip_file_object.namelist()
    logs=[file_name for file_name in files if "mHealth_subject" in file_name]

    # # get our file uris
    # files_uri=glob.glob(dir_to_files+'*.log')

    # Keep track of data
    subjects=[]

    # Read data and put data into subjects and use enumerate to id
    for linear_identifier,file_uri in enumerate(logs):

        # Open the file in read only mode
        # mhealth_file=open(file_uri,'r')
        mhealth_file=zip_file_object.open(file_uri)
        
        # Keep track of file data
        mhealth_data=[]
        # Loop through the file 
        for line in mhealth_file.readlines():
            line=line.decode("utf-8")
            # Get the floating value of each column for a row and append to mhealth_data
            row=list(map(float,line.split()))
            mhealth_data.append(row)        
        mhealth_file.close()
        
        # Build subject
        subject = {
            'id':linear_identifier,
            'data': mhealth_data,
            'file_location':file_uri
        }
        # Append subject
        subjects.append(subject)
    return subjects


## Hyperparameters

In [0]:
# Hyperparameters
# This is how many samples per col
data_length = 2

## Prep the data

In [0]:
# Prep data


# Get the dataset
dataset=mhealth_get_dataset()

# shuffle dataset
random.shuffle(dataset)

# Get 6 training users and 4 test users
training_users,test_data = dataset[:6], dataset[6:]

# These two dictionaries keep track 
training_data_pre  = defaultdict(list) 
test_data_pre =defaultdict(list)

counter=0
previous_label=None
to_put=[]
for user in training_users:
    for user_data in user['data']:
        if not previous_label==user_data[23]:
            counter=1
            to_put=[]
            previous_label=user_data[23]
            user_data.pop()
            to_put+=user_data
        elif previous_label==user_data[23]:
            counter+=1
            user_data.pop()
            to_put+=user_data
        if counter == data_length:
            training_data_pre[previous_label].append(to_put)
            to_put=[]

counter=0
previous_label=None
to_put=[]
for user in test_data:
    for user_data in user['data']:
        if not previous_label==user_data[23]:
            counter=1
            to_put=[]
            previous_label=user_data[23]
            user_data.pop()
            to_put+=user_data
        elif previous_label==user_data[23]:
            counter+=1
            user_data.pop()
            to_put+=user_data
        if counter == data_length:
            test_data_pre[previous_label].append(to_put)
            to_put=[]

training_data,training_labels=[],[]
test_data,test_labels=[],[]

# One hot 
def get_one_hot(targets, nb_classes):
    res = np.eye(nb_classes)[np.array(targets).reshape(-1)]
    return res.reshape(list(targets.shape)+[nb_classes])

for training_label,training in training_data_pre.items():
    for train in training:
        training_labels.append(int(training_label))
        training_data.append(train)

training_data=np.array(training_data)


for training_label,training in test_data_pre.items():
    for train in training:
        test_labels.append(int(training_label))
        test_data.append(train)

test_data=np.array(test_data)

training_labels_len=len(training_labels)
labels=training_labels+test_labels
labels=get_one_hot(np.array(labels),13)
training_labels=labels[:training_labels_len]
test_labels=labels[training_labels_len:]



## Create the model

In [0]:
# Okay lets create our model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(data_length*23), # Our input layer
    tf.keras.layers.Dense(500,activation=tf.keras.activations.relu),
    tf.keras.layers.Dense(500),
    tf.keras.layers.Dense(13,activation=tf.keras.activations.softmax) # Our output layer we have 13 classifcations

])
# Compile our model
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])



## Fit the model

In [8]:
model.fit(training_data,training_labels,validation_data=(test_data,test_labels),epochs=1000)
# -9.2305,3.9479,-4.0236,0.10884,-0.058608,3.4836,-7.467,-8.0885,0.24675,-0.78799,-0.21807,0.19791,18.714,5.4366,2.2399,-5.5648,7.5238,0.47255,-0.25051,1.0172,9.3079,-2.787,-15.309,9
# -8.6677,3.5852,-4.5615,-0.054422,0.22606,-0.75304,-7.1983,-5.9716,0.24675,-0.78799,-0.21807,0.25394,7.7039,9.586,-0.92224,-5.7833,9.6692,0.47255,-0.25051,1.0172,4.6822,1.4321,-15.444,9
# test_array = np.array([[-9.2305,3.9479,-4.0236,0.10884,-0.058608,3.4836,-7.467,-8.0885,0.24675,-0.78799,-0.21807,0.19791,18.714,5.4366,2.2399,-5.5648,7.5238,0.47255,-0.25051,1.0172,9.3079,-2.787,-15.309,-8.6677,3.5852,-4.5615,-0.054422,0.22606,-0.75304,-7.1983,-5.9716,0.24675,-0.78799,-0.21807,0.25394,7.7039,9.586,-0.92224,-5.7833,9.6692,0.47255,-0.25051,1.0172,4.6822,1.4321,-15.444]])

# print(model.predict([test_array])[0][8]*100)

# data_length = 1
# Epoch 100/100
# 157/157 [==============================] - 0s 287us/sample - loss: 0.2621 - acc: 0.9569 - val_loss: 0.6089 - val_acc: 0.9144

Epoch 1/1000
5/5 [==============================] - 0s 48ms/step - loss: 0.8881 - accuracy: 0.2452 - val_loss: 0.8244 - val_accuracy: 0.3838
Epoch 2/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.6918 - accuracy: 0.4194 - val_loss: 0.8137 - val_accuracy: 0.3939
Epoch 3/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.7070 - accuracy: 0.3871 - val_loss: 0.7442 - val_accuracy: 0.3939
Epoch 4/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.6487 - accuracy: 0.4323 - val_loss: 0.7649 - val_accuracy: 0.4747
Epoch 5/1000
5/5 [==============================] - 0s 10ms/step - loss: 0.6783 - accuracy: 0.4452 - val_loss: 0.7388 - val_accuracy: 0.4040
Epoch 6/1000
5/5 [==============================] - 0s 11ms/step - loss: 0.7185 - accuracy: 0.3677 - val_loss: 0.6985 - val_accuracy: 0.4444
Epoch 7/1000
5/5 [==============================] - 0s 12ms/step - loss: 0.6021 - accuracy: 0.5290 - val_loss: 0.6664 - val_accuracy: 0.3939
Epoch 8/1000
